# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Get Data

In [2]:
print(datetime.datetime.now())

Base_Dados = pd.read_parquet('../data/base_consolidada/Base_Consolidada.parquet')

print(datetime.datetime.now())

2023-06-22 08:44:35.703244
2023-06-22 08:44:52.905932


# EDA

In [3]:
type(Base_Dados)

pandas.core.frame.DataFrame

In [4]:
Base_Dados.columns

Index(['Número do Auto', 'Data da Infração (DD/MM/AAAA)', 'Indicador de Abordagem', 'Assinatura do Auto', 'Sentido Trafego', 'UF Infração', 'BR Infração', 'Km Infração', 'Município', 'Indicador Veiculo Estrangeiro', 'UF Placa', 'Descrição Especie Veículo', 'Descrição Marca Veículo', 'Descrição Tipo Veículo', 'Descrição Modelo Veiculo', 'Código da Infração', 'Descrição Abreviada Infração', 'Enquadramento da Infração', 'Início Vigência da Infração', 'Fim Vigência Infração', 'Medição Infração', 'Hora Infração', 'Medição Considerada', 'Excesso Verificado', 'Qtd Infrações'], dtype='object')

In [5]:
Base_Dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4135523 entries, 0 to 4135522
Data columns (total 25 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   Número do Auto                 object
 1   Data da Infração (DD/MM/AAAA)  object
 2   Indicador de Abordagem         object
 3   Assinatura do Auto             object
 4   Sentido Trafego                object
 5   UF Infração                    object
 6   BR Infração                    object
 7   Km Infração                    object
 8   Município                      object
 9   Indicador Veiculo Estrangeiro  object
 10  UF Placa                       object
 11  Descrição Especie Veículo      object
 12  Descrição Marca Veículo        object
 13  Descrição Tipo Veículo         object
 14  Descrição Modelo Veiculo       object
 15  Código da Infração             object
 16  Descrição Abreviada Infração   object
 17  Enquadramento da Infração      object
 18  Início Vigência da Inf

In [6]:
Base_Dados.isnull().sum()

Número do Auto                   0
Data da Infração (DD/MM/AAAA)    0
Indicador de Abordagem           0
Assinatura do Auto               0
Sentido Trafego                  0
UF Infração                      0
BR Infração                      0
Km Infração                      0
Município                        0
Indicador Veiculo Estrangeiro    0
UF Placa                         0
Descrição Especie Veículo        0
Descrição Marca Veículo          0
Descrição Tipo Veículo           0
Descrição Modelo Veiculo         0
Código da Infração               0
Descrição Abreviada Infração     0
Enquadramento da Infração        0
Início Vigência da Infração      0
Fim Vigência Infração            0
Medição Infração                 0
Hora Infração                    0
Medição Considerada              0
Excesso Verificado               0
Qtd Infrações                    0
dtype: int64

In [7]:
Base_Dados.nunique()

Número do Auto                   4135523
Data da Infração (DD/MM/AAAA)        365
Indicador de Abordagem                 2
Assinatura do Auto                     2
Sentido Trafego                        2
UF Infração                           27
BR Infração                          119
Km Infração                         1223
Município                           2078
Indicador Veiculo Estrangeiro          3
UF Placa                              90
Descrição Especie Veículo             14
Descrição Marca Veículo            13173
Descrição Tipo Veículo                24
Descrição Modelo Veiculo           25719
Código da Infração                   378
Descrição Abreviada Infração         378
Enquadramento da Infração            239
Início Vigência da Infração            8
Fim Vigência Infração                  1
Medição Infração                       5
Hora Infração                         24
Medição Considerada                11978
Excesso Verificado                  8518
Qtd Infrações   

In [8]:
Base_Dados.head()

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Indicador Veiculo Estrangeiro,UF Placa,Descrição Especie Veículo,Descrição Marca Veículo,Descrição Tipo Veículo,Descrição Modelo Veiculo,Código da Infração,Descrição Abreviada Infração,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações
0,61eabcb9b7e0679,2022-01-14,C,S,D,MA,230,404,BALSAS,N,MA,ESPECIAL,FIAT/STRADA ENDURANCE CD,CAMINHONETE,STRADA ENDURANCE CD,65991,Conduzir o veículo que não esteja registrado,230 * V,2016-11-01,nan,Nenhuma,16,0,0,1
1,d97a34e658d2e6c,2022-01-30,C,N,D,MA,230,413,BALSAS,N,GO,PASSAGEIRO,VW/GOL 1.0,AUTOMOVEL,VW/GOL 1.0,50100,Dirigir veículo sem possuir CNH/PPD/ACC,162 I,2016-11-01,nan,Nenhuma,17,0,0,1
2,397168b4d993a8f,2022-01-30,C,N,D,MA,230,413,BALSAS,N,GO,PASSAGEIRO,VW/GOL 1.0,AUTOMOVEL,VW/GOL 1.0,51180,Permitir posse/condução do veículo a pessoa se...,164 c/c 162 I,2016-11-01,nan,Nenhuma,17,0,0,1
3,a86157cf8d604d0,2022-01-21,C,N,D,PR,277,31,MORRETES,N,PR,PASSAGEIRO,I/BMW R1200 GS,MOTOCICLETA,I/BMW R1200 GS,65992,Conduzir o veículo registrado que não esteja d...,230 * V,2016-11-01,nan,Nenhuma,12,0,0,1
4,1302ba8ba04bb2f,2022-01-24,C,S,D,MG,40,554,NOVA LIMA,N,RJ,MISTO,I/TOYOTA HILUX SWSRXA4FD,UTILITARIO,I/TOYOTA HILUX SWSRXA4FD,65991,Conduzir o veículo que não esteja registrado,230 * V,2016-11-01,nan,Nenhuma,17,0,0,1


In [9]:
Total_multa = int(Base_Dados.shape[0])
print(f'Total de multas nos 12 meses: {Total_multa}')

Total de multas nos 12 meses: 4135523


In [10]:
# Converter Data
Base_Dados['Data da Infração (DD/MM/AAAA)'] = pd.to_datetime(Base_Dados['Data da Infração (DD/MM/AAAA)'])

In [11]:
# Multas por dia
Multas_Dia = Base_Dados['Data da Infração (DD/MM/AAAA)'].value_counts().sort_index().reset_index()

In [14]:


# Extraíndo mês e dia das datas
Multas_Dia['Mes'] = pd.to_datetime(Multas_Dia['Data da Infração (DD/MM/AAAA)']).dt.month
Multas_Dia['Dia'] = pd.to_datetime(Multas_Dia['Data da Infração (DD/MM/AAAA)']).dt.month

# Renomeando colunas
Multas_Dia.columns = ['Data', 'Quantidade', 'Mes', 'Dia']

In [17]:
Multas_Dia.head()

,Data,Quantidade,Mes,Dia
0,2022-01-01,8533,1,1
1,2022-01-02,13146,1,1
2,2022-01-03,12012,1,1
3,2022-01-04,11248,1,1
4,2022-01-05,11159,1,1


In [18]:
Multas_Dia['Media_Movel'] = Multas_Dia['Quantidade'].rolling(7).mean()

In [19]:
Multas_Dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Data         365 non-null    datetime64[ns]
 1   Quantidade   365 non-null    int64         
 2   Mes          365 non-null    int32         
 3   Dia          365 non-null    int32         
 4   Media_Movel  359 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1)
memory usage: 11.5 KB


# Análise Gráfica

In [21]:
px.line(Multas_Dia, x = 'Data', y = 'Quantidade')

In [22]:
go.Figure(go.Scatter(x = Multas_Dia['Data'], y = Multas_Dia['Quantidade']))

In [23]:
# Sistema Grid

Grid = make_subplots(rows = 1, cols = 2)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Quantidade'), row = 1, col = 1)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Média Móvel'), row = 1, col = 2)

Grid.update_layout(
    title = 'Sistema de Grid',
    showlegend = True,
    legend = dict(
        orientation = 'h',
        yanchor = 'bottom',
        y = 1.02,
        xanchor = 'right',
        x = 1
    )
)

Grid


In [24]:
# Sistema Grid

Grid = make_subplots(rows = 1, cols = 2)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Quantidade'],
    mode = 'lines', name = 'Quantidade'), row = 1, col = 1)

Grid.add_trace(go.Scatter(
    x = Multas_Dia['Data'], 
    y = Multas_Dia['Media_Movel'],
    mode = 'lines', name = 'Média Móvel'), row = 1, col = 1)

Grid.update_layout(
    title = 'Análise de multas diária',
    showlegend = True,
    legend = dict(
        orientation = 'h',
        yanchor = 'bottom',
        y = 1.02,
        xanchor = 'right',
        x = 1
    )
)

Grid


In [27]:
# Boxplot
px.box(Multas_Dia,
       x = 'Mes', 
       y = 'Quantidade',
       color = 'Mes',
       title = 'Distribuição de multas mês')

## Estados

In [28]:
Acumulado_Estados = Base_Dados['UF Infração'].value_counts()
Acumulado_Estados_Perc = Base_Dados['UF Infração'].value_counts(normalize = True)
Acumulado_Estados_Perc_Acum = Base_Dados['UF Infração'].value_counts(normalize = True).cumsum()

# Dict
Dicionario = {
    'Estados': Acumulado_Estados.index,
    'Qtd Multas': Acumulado_Estados.values,
    'Representação': Acumulado_Estados_Perc.values,
    'Acumulado': Acumulado_Estados_Perc_Acum.values
}

Tabela_Estados = pd.DataFrame(Dicionario)

Tabela_Estados.head()

,Estados,Qtd Multas,Representação,Acumulado
0,MG,448672,0.108492,0.108492
1,RJ,418981,0.101313,0.209805
2,BA,417092,0.100856,0.310661
3,SP,378074,0.091421,0.402082
4,RS,252059,0.060950,0.463032


In [29]:
px.funnel(
    Tabela_Estados[Tabela_Estados['Acumulado'] < 0.5],
    y = 'Estados',
    x = 'Qtd Multas',
    title = 'Concentração dos 50%'
)

In [30]:
px.bar(
    Tabela_Estados,
    x = 'Estados',
    y = 'Acumulado',
    title = 'Multas Acumuladas por Estado'
)

## Heatmap

In [31]:
Base_Dados['Mes'] = Base_Dados['Data da Infração (DD/MM/AAAA)'].dt.month

In [32]:
Anl_Estado_Mes = Base_Dados.groupby(by = ['Mes', 'UF Infração']).agg(Quantidade = ('Município', 'count')).reset_index()

In [33]:
Anl_Estado_Mes = Anl_Estado_Mes.pivot_table(index = 'Mes', columns = 'UF Infração', values = 'Quantidade')
Anl_Estado_Mes

UF Infração,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,MG,MS,MT,PA,PB,PE,PI,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
Mes,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1935,4145,1960,1065,29613,11819,3789,9814,12965,6152,25886,19957,15282,4518,8921,9599,6897,28441,47616,5390,7160,931,21648,9393,3347,35375,3320
2,1869,4442,1560,1290,37657,13540,4506,8171,21802,5899,30731,20966,15878,6924,7194,13019,10629,26903,43526,6567,8365,1289,24964,13186,3875,37702,3374
3,1626,3892,1451,1136,38322,12586,7749,9687,26626,7029,38961,22607,15729,6392,7931,14336,8104,25538,45218,7639,9425,1255,23029,14372,3806,34265,3950
4,1178,4175,2049,1670,42969,15011,13708,12860,25979,5965,51360,23967,14066,6836,7465,13093,8156,21734,48050,6544,8750,1523,23871,17591,3901,22893,3168
5,1056,2950,1658,1599,28583,11562,9927,14337,22472,4799,41442,15193,16188,7843,6707,14201,10491,18577,41165,6086,8190,703,19892,17870,3193,21472,3518
6,1199,2133,1780,1048,33048,7611,7566,15296,18386,4014,39351,15812,14364,5398,5165,9545,5515,12865,47713,5028,7654,610,15623,14244,1053,29731,3272
7,1491,1610,1450,876,32852,6897,8809,14789,18359,5540,29557,19015,12435,6650,4797,7816,5585,16983,36345,5192,6754,538,14714,12415,2429,31650,2698
8,1109,2560,1520,1177,34006,10619,3562,15743,17892,7213,37185,19180,13121,6465,6425,9846,6136,16044,22501,6374,9161,907,18305,14365,2660,31345,2702
9,1082,4449,1801,1626,34947,10264,3117,16495,17573,6990,41478,19202,14088,6303,6126,10821,6535,15283,20197,7185,9287,1013,19967,16094,3539,30824,2282


In [34]:
px.imshow(Anl_Estado_Mes, title = 'Mapa de calorMapa de calor | Multas mensais por estado em 2022')

In [35]:
Base_Dados['Enquadramento da Infração'].value_counts(normalize = True).cumsum() * 100

Enquadramento da Infração
218 I                   26.437889
167                     31.968581
203 V                   36.954649
230 * V                 41.899029
218 II                  46.308581
195                     50.183737
230 X                   53.910811
162 I                   57.340801
230 XIII                60.718004
230 XVIII               64.000249
230 IX                  66.831716
193                     68.585956
230, XXIII              70.231939
230 XXII                71.768794
162 V                   73.163201
230 XVI                 74.502620
250 I e                 75.807365
230 VI                  77.045443
165-A                   78.223190
252 IV                  79.217671
252,§ unico             80.138812
165-B                   81.054996
164 c/c 162 I           81.950046
168                     82.739257
244 I                   83.368367
244 X                   83.989329
202 I                   84.570368
185 II                  85.097532
230 VII               

Não há o valor das multas na base de dados, porém o enquadramento e no site do DETRAN, há essa informação. Vamos fazer Web Scrapping para conseguir estes dados.

# Web Scrapping

In [47]:
Url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao='

pd.read_html(Url)[0].head()

,Código,Desd.,Infração,Artigo,Infrator,Pts,Valor
0,5622,4,PARAR NOS CANTEIROS CENTRAIS OU DIVISORES DE P...,182 * VI,CON,3,8838
1,5622,5,PARAR NAS MARCAS DE CANALIZACAO,182 * VI,CON,3,8838
2,5630,0,PARAR NA AREA DE CRUZAMENTO DE VIAS,182 * VII,CON,4,13016
3,5649,1,PARAR NOS VIADUTOS,182 * VIII,CON,4,13016
4,5649,2,PARAR NAS PONTES,182 * VIII,CON,4,13016


In [48]:
# Loop no site do Detran e buscar os dados
Url = 'https://www.detran.mg.gov.br/infracoes/consultar-tipos-infracoes/index/index/index/index/index/index/index/index/index/index/index/index/index/lista-de-infracoes?artigo=&descricao=&page='

# Tabela vazia
Base_Consolidada = pd.DataFrame()

for Loop in range(1, 24):
    
    # Construindo a URL
    Link = f'{Url}{Loop}'
    
    # Lendo os dados da Web
    Dados_web = pd.read_html(Link)[0]
    
    # Consolidar
    Base_Consolidada = pd.concat([Base_Consolidada, Dados_web])


In [49]:
Base_Consolidada.shape

(442, 7)

In [50]:
Base_Consolidada.head()

,Código,Desd.,Infração,Artigo,Infrator,Pts,Valor
0,5622,4,PARAR NOS CANTEIROS CENTRAIS OU DIVISORES DE P...,182 * VI,CON,3,8838
1,5622,5,PARAR NAS MARCAS DE CANALIZACAO,182 * VI,CON,3,8838
2,5630,0,PARAR NA AREA DE CRUZAMENTO DE VIAS,182 * VII,CON,4,13016
3,5649,1,PARAR NOS VIADUTOS,182 * VIII,CON,4,13016
4,5649,2,PARAR NAS PONTES,182 * VIII,CON,4,13016


In [65]:
Base_Consolidada['Valor'] = pd.to_numeric(Base_Consolidada['Valor'], errors='coerce')
Base_Consolidada = Base_Consolidada.dropna(subset=['Valor'])


In [61]:
Base_Consolidada['Valor'] = Base_Consolidada['Valor'] / 100

In [62]:
Base_Consolidada.rename(columns = {'Código': 'Código da Infração'}, inplace = True)

In [ ]:
Base_Consolidada['Código da Infração'] = Base_Consolidada['Código da Infração'].astype('int64')

In [67]:
Tab_Preco = Base_Consolidada.groupby(by=['Código da Infração'])['Valor'].apply(lambda x: np.mean(x.astype(float))).reset_index()

In [69]:
# Reencresver a coluna de Infração
Base_Dados['Código da Infração'] = Base_Dados['Código da Infração'].apply( lambda Loop : Loop[0:4] )

In [70]:
# Converto para numero o codigo
Base_Dados['Código da Infração'] = pd.to_numeric( Base_Dados['Código da Infração'], errors = 'coerce')

In [72]:
Base_Dados['Código da Infração'].dtypes

dtype('int64')

In [73]:
Tab_Preco['Código da Infração'].dtypes

dtype('int64')

In [74]:
# Cruzando os dados
Cruzamento = pd.merge( Base_Dados, Tab_Preco, on='Código da Infração', how='left' )
Cruzamento.head()

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Sentido Trafego,UF Infração,BR Infração,Km Infração,Município,Indicador Veiculo Estrangeiro,UF Placa,Descrição Especie Veículo,Descrição Marca Veículo,Descrição Tipo Veículo,Descrição Modelo Veiculo,Código da Infração,Descrição Abreviada Infração,Enquadramento da Infração,Início Vigência da Infração,Fim Vigência Infração,Medição Infração,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações,Mes,Valor
0,61eabcb9b7e0679,2022-01-14,C,S,D,MA,230,404,BALSAS,N,MA,ESPECIAL,FIAT/STRADA ENDURANCE CD,CAMINHONETE,STRADA ENDURANCE CD,6599,Conduzir o veículo que não esteja registrado,230 * V,2016-11-01,nan,Nenhuma,16,0,0,1,1,293.47
1,d97a34e658d2e6c,2022-01-30,C,N,D,MA,230,413,BALSAS,N,GO,PASSAGEIRO,VW/GOL 1.0,AUTOMOVEL,VW/GOL 1.0,5010,Dirigir veículo sem possuir CNH/PPD/ACC,162 I,2016-11-01,nan,Nenhuma,17,0,0,1,1,880.41
2,397168b4d993a8f,2022-01-30,C,N,D,MA,230,413,BALSAS,N,GO,PASSAGEIRO,VW/GOL 1.0,AUTOMOVEL,VW/GOL 1.0,5118,Permitir posse/condução do veículo a pessoa se...,164 c/c 162 I,2016-11-01,nan,Nenhuma,17,0,0,1,1,880.41
3,a86157cf8d604d0,2022-01-21,C,N,D,PR,277,31,MORRETES,N,PR,PASSAGEIRO,I/BMW R1200 GS,MOTOCICLETA,I/BMW R1200 GS,6599,Conduzir o veículo registrado que não esteja d...,230 * V,2016-11-01,nan,Nenhuma,12,0,0,1,1,293.47
4,1302ba8ba04bb2f,2022-01-24,C,S,D,MG,40,554,NOVA LIMA,N,RJ,MISTO,I/TOYOTA HILUX SWSRXA4FD,UTILITARIO,I/TOYOTA HILUX SWSRXA4FD,6599,Conduzir o veículo que não esteja registrado,230 * V,2016-11-01,nan,Nenhuma,17,0,0,1,1,293.47


In [75]:
# Analise por UF e Preço
Tab_Soma = Cruzamento.groupby(by='UF Infração').agg(
    {'Valor':['count', 'sum']},
)

Tab_Soma.columns = Tab_Soma.columns.droplevel()

Tab_Soma = Tab_Soma.reset_index()

Tab_Soma.head()

,UF Infração,count,sum
0,AC,14520,3.815801e+06
1,AL,37570,1.001225e+07
2,AM,18356,4.812791e+06
3,AP,16825,5.376411e+06
4,BA,374633,7.648385e+07


In [76]:
px.scatter( 
    Tab_Soma, 
    x='count', 
    y='sum', 
    color='UF Infração', 
    size='count', 
    log_x=True, 
    size_max=60, 
    title='Bubble PLOT' )

In [77]:
fig = px.scatter( Tab_Soma, x='count', y='sum', color='UF Infração', title='Scatter PLOT')
fig.update_traces(marker=dict(size=12) )

In [78]:
Analise_Valor_Mes = Cruzamento.groupby( by=['Mes'] )['Valor'].sum().reset_index()
Analise_Valor_Mes

,Mes,Valor
0,1,7.134070e+07
1,2,8.340845e+07
2,3,8.640171e+07
3,4,8.685282e+07
4,5,7.579054e+07
5,6,6.787070e+07
6,7,6.424681e+07
7,8,6.834306e+07
8,9,7.236469e+07
9,10,7.930858e+07
